In [ ]:
# THESE ARE PLACE HOLDERS PLEASE MAKE SURE THE INFORMATION MATCHES YOUR SEARCH
API_KEY = 'YOUR_API_KEY' # Replace with your actual key
database_code = 'WOK'
page_limit = 50
page_max = 5
sort_field = 'LD'
ascending = 'true' # or 'false' or ''
base_url = 'https://api.clarivate.com/apis/wos-starter/v1/documents'
search_phrase = '"climate change" OR "global warming" AND sustainability'

In [13]:
import urllib.parse
import requests
import json
import csv

In [8]:
def create_encoded_query(phrase):
    """Encodes a string for use in a URL."""
    return urllib.parse.quote(phrase)


In [9]:
search_phrase_with_tag = f'TS=({search_phrase})'

params = {
    'db': database_code,
    'q': search_phrase_with_tag,
    'limit': page_limit
}

if ascending == 'true':
    params['sortField'] = f'{sort_field} A' # Note: API docs use a space
elif ascending == 'false':
    params['sortField'] = f'{sort_field} D'
else:
    params['sortField'] = sort_field


In [10]:
headers = {
    'accept': 'application/json',
    'X-ApiKey': API_KEY
}

all_results = []


In [11]:
for i in range(1, page_max + 1):
  # Set the page for the current iteration
  params['page'] = i
  
  print(f"Requesting page {i}...")
  
  # Make the API call
  response = requests.get(base_url, headers=headers, params=params)
  
  # Check if the request was successful
  if response.status_code == 200:
      data = response.json()
      # The list of documents is in the 'hits' key of the JSON response
      page_hits = data.get('hits', [])
      if page_hits:
          # Add the results from this page to our master list
          all_results.extend(page_hits)
          print(f"  ... Success! Added {len(page_hits)} results.")
      else:
          # Stop if a page returns no results
          print("  ... No more results found. Stopping.")
          break 
  else:
      print(f"  ... Error on page {i}: Status code {response.status_code}")
      print(f"  ... Response: {response.text}")
      # Stop the loop if an error occurs
      break

Requesting page 1...
  ... Success! Added 50 results.
Requesting page 2...
  ... Success! Added 50 results.
Requesting page 3...
  ... Success! Added 50 results.
Requesting page 4...
  ... Success! Added 50 results.
Requesting page 5...
  ... Success! Added 50 results.


In [14]:
# Define the name of the output CSV file
csv_file_name = 'wos_results.csv'

# Check if we have any results to write
if all_results:
    # Open the file in write mode
    with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
        # The 'all_results' list contains dictionaries. We can use the keys
        # of the first dictionary as the headers for our CSV.
        headers = all_results[0].keys()
        
        # Create a DictWriter object which maps dictionaries to CSV rows
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        
        # Write the header row
        writer.writeheader()
        
        # Write all the dictionary rows from our results list
        writer.writerows(all_results)
        
    print(f"\nSuccessfully saved {len(all_results)} results to {csv_file_name}")
else:
    print("\nNo results to save to CSV.")


Successfully saved 250 results to wos_results.csv
